In [1]:
from pyspark.sql import SparkSession
import os

In [51]:
from pyspark.sql.functions import input_file_name, regexp_extract
from pyspark.sql.types import StringType
def custom_formatter(s: str):
    index = -1;
    while s[index] != '/':
        index -= 1
    index -= 1
    print(s[index])
    location = []
    while s[index] != '/':
        location.insert(0, s[index])
        index -= 1
    location = "".join(location)
    return location

spark.udf.register("custom_formatter", custom_formatter, StringType())

25/09/02 08:41:21 WARN SimpleFunctionRegistry: The function custom_formatter replaced a previously registered function.


<function __main__.custom_formatter(s: str)>

In [2]:
spark = SparkSession.builder \
    .appName("S3JsonReader") \
    .config("spark.hadoop.fs.s3a.connection.timeout", "60000") \
    .config("spark.hadoop.fs.s3a.socket.timeout", "60000") \
    .config("spark.hadoop.fs.s3a.connection.establish.timeout", "60000") \
    .config("spark.hadoop.fs.s3a.threads.keepalivetime", "60000") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.multipart.purge.age", "24") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "com.amazonaws.auth.EnvironmentVariableCredentialsProvider") \
    .getOrCreate()

25/09/02 13:20:05 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
hadoop_conf = spark.sparkContext._jsc.hadoopConfiguration()
iterator = hadoop_conf.iterator()
while iterator.hasNext():
    entry = iterator.next()
    if "fs.s3a" in entry.getKey():
        print(f"{entry.getKey()}: {entry.getValue()}")

fs.s3a.select.output.csv.record.delimiter: \n
fs.s3a.select.input.csv.quote.character: "
fs.s3a.path.style.access: false
fs.s3a.select.input.compression: none
fs.s3a.max.total.tasks: 32
fs.s3a.vectored.read.min.seek.size: 128K
fs.s3a.select.output.csv.quote.fields: always
fs.s3a.vectored.read.max.merged.size: 2M
fs.s3a.select.input.csv.header: none
fs.s3a.impl: org.apache.hadoop.fs.s3a.S3AFileSystem
fs.s3a.select.enabled: true
fs.s3a.committer.staging.tmp.path: tmp/staging
fs.s3a.executor.capacity: 16
fs.s3a.multiobjectdelete.enable: true
fs.s3a.multipart.purge: false
fs.s3a.connection.establish.timeout: 30s
fs.s3a.downgrade.syncable.exceptions: true
fs.s3a.multipart.purge.age: 24h
fs.s3a.audit.enabled: true
fs.s3a.select.input.csv.record.delimiter: \n
fs.s3a.accesspoint.required: false
fs.s3a.retry.limit: 7
fs.s3a.committer.name: file
fs.s3a.threads.max: 96
fs.s3a.threads.keepalivetime: 60s
fs.s3a.select.input.csv.quote.escape.character: \\
fs.s3a.multipart.threshold: 128M
fs.s3a.ssl.

In [4]:
hadoop_conf = spark.sparkContext._jsc.hadoopConfiguration()
hadoop_conf.clear()  # Clear all Hadoop configurations
hadoop_conf.set("fs.s3a.aws.credentials.provider", "com.amazonaws.auth.EnvironmentVariableCredentialsProvider")
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.connection.timeout", "60000")
hadoop_conf.set("fs.s3a.socket.timeout", "60000")
hadoop_conf.set("fs.s3a.connection.establish.timeout", "60000")
hadoop_conf.set("fs.s3a.threads.keepalivetime", "60")
hadoop_conf.set("fs.s3a.multipart.purge.age", "24")

In [54]:
df = spark.read.json("s3a://af-weather-lake-01/data/weather/")

In [55]:
current_df = df.select("current.cloud_cover", "date", "latitude").show()

+-----------+---------------+--------+
|cloud_cover|           date|latitude|
+-----------+---------------+--------+
|        100|2025-09-02_0400|  23.125|
|        100|2025-09-01_0800|  23.125|
|        100|2025-09-01_0100|  23.125|
|        100|2025-09-02_1300|  23.125|
|        100|2025-09-01_0000|  23.125|
|        100|2025-09-01_1400|  23.125|
|        100|2025-09-01_2000|  23.125|
|        100|2025-09-02_0500|  23.125|
|        100|2025-09-02_0200|  23.125|
|         87|2025-09-01_0500|  23.125|
|        100|2025-09-01_0900|  23.125|
|        100|2025-09-01_1000|  23.125|
|        100|2025-08-31_2200|  23.125|
|        100|2025-09-01_0700|  23.125|
|        100|2025-09-01_0500|    25.0|
|        100|2025-09-02_0400|    25.0|
|        100|2025-09-01_0400|  23.125|
|        100|2025-09-01_0600|  23.125|
|        100|2025-09-01_1300|    25.0|
|        100|2025-09-01_1700|    25.0|
+-----------+---------------+--------+
only showing top 20 rows


In [8]:
import pyspark.sql.functions as F

count = df.select("*")\
    .agg(
        F.count("*").alias("num_rows")
    ) \
    .show()

[Stage 4:>                                                        (0 + 10) / 10]

+--------+
|num_rows|
+--------+
|     120|
+--------+



In [60]:
new_df = df.select("current.*", "date", "latitude", "longitude") \
    .withColumn("file_path", input_file_name()) \
    .withColumn("city", regexp_extract("file_path", r"date=[^/]+/([^/]+)/[^/]+\.json$", 1))

In [67]:
reordered_df = new_df.select("*") \
    .show()

+--------------------+-----------+--------+------+-------------+------------+----+--------------------+-------+----------------+--------------+----------------+------------+------------------+--------------+--------------+---------------+--------+---------+--------------------+------+
|apparent_temperature|cloud_cover|interval|is_day|precipitation|pressure_msl|rain|relative_humidity_2m|showers|surface_pressure|temperature_2m|            time|weather_code|wind_direction_10m|wind_gusts_10m|wind_speed_10m|           date|latitude|longitude|           file_path|  city|
+--------------------+-----------+--------+------+-------------+------------+----+--------------------+-------+----------------+--------------+----------------+------------+------------------+--------------+--------------+---------------+--------+---------+--------------------+------+
|                91.1|        100|     900|     1|          0.1|      1003.0| 0.0|                  83|    0.1|           946.8|          81.1

In [ ]:
# s3://af-weather-lake-01/data/weather/date=2025-09-01_0300/miami/miami_2025-09-01_0300.json